In [13]:
import cv2
import os
import time
import torch
import numpy as np
from torch import nn
from torchvision import models

In [2]:
def capture(filename, timesep, rgb, h, w):
  frames = np.zeros((timesep, rgb, h, w), dtype=float)
  i = 0
  vc = cv2.VideoCapture(filename)
  if vc.isOpened():
    rval, frame = vc.read()
    while rval and i < timesep:
      # Resize the frame to the specified width and height
      frm = cv2.resize(frame, (w, h))
      frm = np.expand_dims(frm, axis=0)
      frm = np.moveaxis(frm, -1, 1)
      if np.max(frm) > 1:
        frm = frm / 255.0
      frames[i][:] = frm
      i += 1
      rval, frame = vc.read()
    vc.release()  # Release the video capture object
  else:
    print("Error: Unable to open video file")
  return frames

In [14]:
class TimeWarp(nn.Module):
  '''video to frames'''

  def __init__(self, baseModel, method='sqeeze'):
    super(TimeWarp, self).__init__()
    self.baseModel = baseModel
    self.method = method

  def forward(self, x):
    batch_size, time_steps, C, H, W = x.size()
    if self.method == 'loop':
      output = []
      for i in range(time_steps):
        # input one frame at a time into the basemodel
        x_t = self.baseModel(x[:, i, :, :, :])
        # Flatten the output
        x_t = x_t.view(x_t.size(0), -1)
        output.append(x_t)
      # end loop
      # make output as  ( samples, timesteps, output_size)
      x = torch.stack(output, dim=0).transpose_(0, 1)
      output = None  # clear var to reduce data  in memory
      x_t = None  # clear var to reduce data  in memory
    else:
      # reshape input  to be (batch_size * timesteps, input_size)
      x = x.contiguous().view(batch_size * time_steps, C, H, W)
      x = self.baseModel(x)
      x = x.view(x.size(0), -1)
      # make output as  ( samples, timesteps, output_size)
      x = x.contiguous().view(batch_size, time_steps, x.size(-1))
    return x

In [4]:
class extractlastcell(nn.Module):
  def forward(self, x):
    out, _ = x
    return out[:, -1, :]

In [22]:
def Model(weight='model_weight.pth'):
  # Create model
  num_classes = 1
  dr_rate = 0.2
  pretrained = True
  rnn_hidden_size = 80
  rnn_num_layers = 1
  baseModel = models.vgg19(pretrained=pretrained).features

  i = 0
  for child in baseModel.children():
    if i < 28:
      for param in child.parameters():
        param.requires_grad = False
    else:
      for param in child.parameters():
        param.requires_grad = True
    i += 1

  num_features = 12800
  # Example of using Sequential
  model = nn.Sequential(
      TimeWarp(baseModel, method='loop'),
      nn.LSTM(num_features, rnn_hidden_size, rnn_num_layers,
              batch_first=True, bidirectional=True),
      extractlastcell(),
      nn.Linear(160, 256),
      nn.ReLU(),
      nn.Dropout(dr_rate),
      nn.Linear(256, num_classes)
  )
  checkpoint = torch.load(weight, map_location=torch.device('cpu'))
  model.load_state_dict(checkpoint['state_dict'])
  return model

In [17]:
def pred_fight(model, video, acuracy=0.58):
  model.eval()
  outputs = model(video)
  torch.sigmoid(outputs)
  preds = torch.sigmoid(outputs).to('cpu')
  preds = preds.detach().numpy()
  if preds[0][0] >= acuracy:
    return True, preds[0][0]
  else:
    return False, preds[0][0]

In [ ]:
np.random.seed(1234)
device = torch.device("cpu")   # Use CPU only
model22 = Model()
model22.to(device)

In [29]:
def main_fight(video_path, accuracyfight=0.65):
  tmp_obj = {}

  if os.path.exists('./tmp.mp4'):
    os.remove('./tmp.mp4')

  # Open the local video file
  cap = cv2.VideoCapture(video_path)

  # Check if the video file was opened successfully
  if not cap.isOpened():
    print("Error: Unable to open video file")
    return None

  else:
    # Open a file to write the video data
    with open("tmp.mp4", "wb") as file:
      while True:
        # Read a frame from the video
        ret, frame = cap.read()
        # Check if the frame was read successfully
        if not ret:
          break  # Break the loop if no more frames are available
        # Write the frame data to the file
        file.write(frame)

  vid = capture("tmp.mp4", 40, 3, 170, 170)
  v = np.array([vid])
  v = torch.from_numpy(v)
  v = v.to(device, dtype=torch.float)
  millis = int(round(time.time() * 1000))
  f, precent = pred_fight(model22, v, acuracy=accuracyfight)
  tmp_obj = {'fight': f, 'precentegeoffight': str(precent)}
  millis2 = int(round(time.time() * 1000))
  tmp_obj['processing_time'] = str((millis2-millis)/1000)
  return tmp_obj

In [12]:
# main_fight(video_path='give-your-video-path-to-predict-violence')

In [30]:
main_fight(video_path="C:/Users/k25ha/OneDrive/Desktop/video_20240122_192851.mp4")

Error: Unable to open video file


{'fight': True, 'precentegeoffight': '0.66339463', 'processing_time': '5.582'}